# Analisis de Sentimientos Foros Moodle Deep Learning Modelo 1

##Importar las librerias necesarias

In [ ]:
#!python -m spacy download es_core_news_sm
#!pip install fasttext
%matplotlib inline
from matplotlib import pyplot as plt
import csv
import warnings
import lxml.html.clean
warnings.filterwarnings('ignore')
import re
from re import sub
import multiprocessing
from textblob import TextBlob
import seaborn as sns
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import FreqDist
from bs4 import BeautifulSoup
import re, string, unicodedata
import spacy
from sklearn.metrics import accuracy_score

from nltk.tokenize.toktok import ToktokTokenizer
import collections

from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

import logging
from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import keras

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.layers.embeddings import Embedding

from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt')

nltk.download('stopwords') 
stop_words =stopwords.words('spanish')
#nlp = spacy.load('es_core_news_sm')
stemmer = SnowballStemmer("spanish")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importar Conjunto de Datos Consolidado

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

df= pd.read_csv('/content/gdrive/My Drive/TFM_Analisis_Sentimientos/Corpus/dataset_balanced.csv', encoding='utf-8')

Mounted at /content/gdrive/


# Pre procesamiento

In [ ]:
#Funciones para limpiar el texto
#!pip install spacy
#!python -m spacy download es_core_news_sm

stop_words.extend(['si', 'tambien', 'asi', 'debe', 'tener','cada','ademas','parte','ser','ma','mismo','mas'])  

def processText(raw_text, remove_stopwords=True, stemming=False, split_text=False):

    if remove_stopwords: # Eliminar palabras vacias
        stops = set(stopwords.words("spanish"))
        words = [w for w in words if not w in stops]
          
    if split_text==True:  # split text
        return (words)    
    
    return( " ".join(words)) 


def sentiment_num(sentiment):
  if sentiment == 'POSITIVO':
    return 1
  elif sentiment == 'NEGATIVO':
    return -1
  elif sentiment == 'NEUTRAL':
    return 0  



In [ ]:
df.head(8)

,text,sentiment
0,curso muy interesante en cuanto a contenido y en cuanto a desarrollo personal,POSITIVO
1,el mejor saludos desde panama,POSITIVO
2,es estupenda la manera en que te ensenan muy didactico y claro le doy un,POSITIVO
3,me ha parecido un curso fantastico breve didactico y muy bien preparado¡felicidadesfrancisco eugenio,POSITIVO
4,primera replica bueno creo que falta organizar la ideas pero al menos se miro el video de forma rapida en saltos como de cierta forma se responde con alguna ideas bastantes cambios que presenta ofrece distintas fases cambiante e incluso alterable teorias cientificas flexibles que se permiten adaptar etc lo que no comprendo son las siguientes frases investigaciones beneficiosas y lucrativas para el crecimiento de estas ciencias ciencia complejo ingenioso conocimiento para pasar de algo antiguo a algo innovador la ciencia trata de preguntar constantemente el porque de las cosas completamente redimidas esto es inedito en seguir buscando y este siga siendo persistente considero que ademas no se revisa lo que pide la pregunta que al plano de relatar lo presenta el conferencista tambien cosiste en relatar la experiencia personal de la cual no ha y nada en esta primera replica,POSITIVO
5,es muy claro que la pelicula es una recopilacion de hechos historicos en los cuales representan varios temas tratados en clase como la desviacion en la que un lider cambia su proposito o los medios manipulan la informacion para conveniencia se establece que la institucion es el estado que gobierna con unas leyes a conveniencia y lleva a un fanatismos con simbolizado con una imitacion por la esvastica utilizada por hitler por otro lado es un llamado a la revolucion necesaria asi conocida desde la revolucion francesa que se ha visto reflejada en muchas ocasiones con el fin de proteger una vez mas al pueblo oprimido y atacar el sistema que beneficia a unos cuantos en un punto de vista personal senti que esa pelicula solo cambio el contexto historico y llevo a las pantallas para demostrar el lema muy conocido el exceso del poder corrompe ya que aunque la idea de revolucion es necesaria no debe manejarse de esa forma,POSITIVO
6,excelente curso muy enriquecedor,POSITIVO
7,excelente curso todos los aplausos y felicitaciones para el docente excelente y explicito el material que nos brindo muy completo la verdad quedo mas que feliz y agradecida con el docente y la universidad,POSITIVO


In [ ]:
df['sentiment'] = df['sentiment'].apply(sentiment_num)

##Dividir en datos de entrenamiento y test

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], \
                                                    test_size=0.1, random_state=0)

In [ ]:

X_train_cleaned = []
X_test_cleaned = []

for d in X_train:
    X_train_cleaned.append(processText(d))

for d in X_test:
    X_test_cleaned.append(processText(d))

UnboundLocalError: ignored

In [ ]:
# Split review text into parsed sentences uisng NLTK's punkt tokenizer
# nltk.download()
tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

def parseSent(review, tokenizer, remove_stopwords=False):
    '''
    Parse text into sentences
    '''
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleanText(raw_sentence, remove_stopwords, split_text=True))
    return sentences


# Parse each review in the training set into sentences
sentences = []
for review in X_train_cleaned:
    sentences += parseSent(review, tokenizer)
    
print('%d parsed sentence in the training set\n'  %len(sentences))
print('Show a parsed sentence in the training set : \n',  sentences[10])

8694 parsed sentence in the training set

Show a parsed sentence in the training set : 
 ['cual', 'de', 'todas', 'las', 'herramientas', 'mencionadas', 'fue', 'la', 'que', 'mas', 'le', 'llamo', 'la', 'atencionrespuestala', 'herramienta', 'que', 'mas', 'me', 'llamo', 'la', 'atencion', 'fue', 'el', 'biofeedback', 'debido', 'a', 'que', 'esta', 'permite', 'que', 'el', 'paciente', 'con', 'ayuda', 'y', 'supervision', 'del', 'terapeuta', 'es', 'consciente', 'de', 'algunos', 'procesos', 'fisiologicos', 'de', 'los', 'cuales', 'normalmente', 'no', 'tiene', 'en', 'cuenta', 'para', 'indentificarlos', 'y', 'posteriormente', 'controlarlos', 'se', 'incia', 'colocando', 'sensores', 'al', 'paciente', 'los', 'cuales', 'no', 'son', 'invasivos', 'y', 'permite', 'recoger', 'una', 'senal', 'que', 'se', 'presentara', 'en', 'la', 'pantalla', 'de', 'un', 'computador', 'para', 'que', 'el', 'paciente', 'evidencie', 'aquellas', 'alteraciones', 'dependiendo', 'su', 'caso', 'y', 'por', 'medio', 'del', 'psicologo', '

##Entrenar al modelo Word2Vec

In [ ]:
num_features = 300  #embedding dimension                     
min_word_count = 10                
num_workers = 4       
context = 10                                                                                          
downsampling = 1e-3 

print("Training Word2Vec model ...\n")
w2v = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count,\
                 window = context, sample = downsampling)
w2v.init_sims(replace=True)
w2v.save("w2v_300features_10minwordcounts_10context") #save trained word2vec model

print("Number of words in the vocabulary list : %d \n" %len(w2v.wv.index2word)) #4016 
print("Show first 10 words in the vocalbulary list  vocabulary list: \n", w2v.wv.index2word[0:10])

Training Word2Vec model ...

Number of words in the vocabulary list : 5285 

Show first 10 words in the vocalbulary list  vocabulary list: 
 ['de', 'la', 'que', 'y', 'el', 'en', 'a', 'los', 'se', 'es']


In [ ]:
# Transfrom the training data into feature vectors

def makeFeatureVec(review, model, num_features):
    '''
    Transform a review to a feature vector by averaging feature vectors of words 
    appeared in that review and in the volcabulary list created
    '''
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.wv.index2word) #index2word is the volcabulary list of the Word2Vec model
    isZeroVec = True
    for word in review:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
            isZeroVec = False
    if isZeroVec == False:
        featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    '''
    Transform all reviews to feature vectors using makeFeatureVec()
    '''
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
        counter = counter + 1
    return reviewFeatureVecs

In [ ]:
# Get feature vectors for training set
X_train_cleaned = []
for review in X_train:
    X_train_cleaned.append(cleanText(review, remove_stopwords=True, split_text=True))
trainVector = getAvgFeatureVecs(X_train_cleaned, w2v, num_features)
print("Training set : %d feature vectors with %d dimensions" %trainVector.shape)


# Get feature vectors for validation set
X_test_cleaned = []
for review in X_test:
    X_test_cleaned.append(cleanText(review, remove_stopwords=True, split_text=True))
testVector = getAvgFeatureVecs(X_test_cleaned, w2v, num_features)
print("Validation set : %d feature vectors with %d dimensions" %testVector.shape)


Training set : 8694 feature vectors with 300 dimensions
Validation set : 966 feature vectors with 300 dimensions


In [ ]:
# Load trained Word2Vec model
w2v = Word2Vec.load("w2v_300features_10minwordcounts_10context")


# Get Word2Vec embedding matrix
embedding_matrix = w2v.wv.syn0  # embedding matrix, type = numpy.ndarray 
print("Shape of embedding matrix : ", embedding_matrix.shape) #(4016, 300) = (volcabulary size, embedding dimension)
# w2v.wv.syn0[0] #feature vector of the first word in the volcabulary list

Shape of embedding matrix :  (5285, 300)


In [ ]:
w2v.most_similar(positive=['excelente'], topn = 10)

[('finanzas', 0.9381696581840515),
 ('programacion', 0.9275283813476562),
 ('completo', 0.926627516746521),
 ('r', 0.9260179400444031),
 ('recomendado', 0.917519211769104),
 ('practico', 0.9111957550048828),
 ('introduccion', 0.894296407699585),
 ('basico', 0.889096736907959),
 ('excel', 0.888027012348175),
 ('buen', 0.8877347707748413)]

In [ ]:
print(df["sentiment"].value_counts())

 1    3220
-1    3220
 0    3220
Name: sentiment, dtype: int64


In [ ]:
top_words = embedding_matrix.shape[0] #4016
maxlen = 100 
batch_size = 32
nb_classes = 3
nb_epoch = 3


# Vectorize X_train and X_test to 2D tensor
tokenizer = Tokenizer(nb_words=top_words) #only consider top 20000 words in the corpse
tokenizer.fit_on_texts(X_train)
# tokenizer.word_index #access word-to-index dictionary of trained tokenizer

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

X_train_seq = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test_seq = sequence.pad_sequences(sequences_test, maxlen=maxlen)


# one-hot encoding of y_train and y_test
y_train_seq = np_utils.to_categorical(y_train, nb_classes)
y_test_seq = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train_seq.shape) #(27799, 100)
print('X_test shape:', X_test_seq.shape) #(3089, 100)
print('y_train shape:', y_train_seq.shape) #(27799, 2)
print('y_test shape:', y_test_seq.shape) #(3089, 2)

X_train shape: (8694, 100)
X_test shape: (966, 100)
y_train shape: (8694, 3)
y_test shape: (966, 3)


In [ ]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
import pickle
from keras.callbacks import ModelCheckpoint

# Construct Word2Vec embedding layer
embedding_layer = Embedding(embedding_matrix.shape[0], #4016
                            embedding_matrix.shape[1], #300
                            weights=[embedding_matrix])

model1 = Sequential()
model1.add(embedding_layer)
model1.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model1.add(layers.Dense(3,activation='softmax'))
model1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 300)         1585500   
                                                                 
 bidirectional_5 (Bidirectio  (None, 40)               51360     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 3)                 123       
                                                                 
Total params: 1,636,983
Trainable params: 1,636,983
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile model
model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history1= model1.fit(X_train_seq, y_train_seq,validation_data=(X_test_seq, y_test_seq), epochs=70, callbacks=[checkpoint1])

Epoch 1/70
268/272 [============================>.] - ETA: 0s - loss: 0.5327 - accuracy: 0.5652
Epoch 1: val_accuracy improved from -inf to 0.68219, saving model to best_model1.hdf5
272/272 [==============================] - 7s 14ms/step - loss: 0.5317 - accuracy: 0.5671 - val_loss: 0.4580 - val_accuracy: 0.6822
Epoch 2/70
269/272 [============================>.] - ETA: 0s - loss: 0.4051 - accuracy: 0.7276
Epoch 2: val_accuracy did not improve from 0.68219
272/272 [==============================] - 3s 11ms/step - loss: 0.4062 - accuracy: 0.7259 - val_loss: 0.4544 - val_accuracy: 0.6304
Epoch 3/70
268/272 [============================>.] - ETA: 0s - loss: 0.3783 - accuracy: 0.7517
Epoch 3: val_accuracy improved from 0.68219 to 0.70083, saving model to best_model1.hdf5
272/272 [==============================] - 3s 11ms/step - loss: 0.3779 - accuracy: 0.7520 - val_loss: 0.4389 - val_accuracy: 0.7008
Epoch 4/70
268/272 [============================>.] - ETA: 0s - loss: 0.3116 - accuracy: 0

#Evaluar al modelo

In [ ]:
best_model = keras.models.load_model("best_model1.hdf5")

NameError: ignored

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

print(history.history.keys())
# history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import time
import numpy as np

predictions = best_model.predict(X_test)
start_time = time.time()
test_predictions = np.argmax(best_model.predict(X_test), axis=-1)
print(classification_report(y_test.argmax(axis=1),test_predictions))
print("Time taken to predict the model " + str(time.time() - start_time))

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions, decimals=0).argmax(axis=1))

import seaborn as sns
conf_matrix = pd.DataFrame(matrix, index = ['Neutral','Negative','Positive'],columns = ['Neutral','Negative','Positive'])
#Normalizing
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize = (15,15))
sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 15})

In [ ]:
# Construct Word2Vec embedding layer
embedding_layer = Embedding(embedding_matrix.shape[0], #4016
                            embedding_matrix.shape[1], #300
                            weights=[embedding_matrix])


# Construct LSTM with Word2Vec embedding
model2 = Sequential()
model2.add(embedding_layer)
model2.add(LSTM(128, recurrent_dropout =0.2)) 
model2.add(Dropout(0.2))
model2.add(Dense(nb_classes))
model2.add(Activation('softmax'))
model2.summary()

# Compile model
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model2.fit(X_train_seq, y_train_seq, batch_size=batch_size, epochs=70, verbose=1)


# Model evaluation
score = model2.evaluate(X_test_seq, y_test_seq, batch_size=batch_size)
print('Test loss : {:.4f}'.format(score[0]))
print('Test accuracy : {:.4f}'.format(score[1]))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         1541100   
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
 activation (Activation)     (None, 3)                 0         
                                                                 
Total params: 1,761,135
Trainable params: 1,761,135
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
272/272 [===========================

KeyboardInterrupt: ignored

In [ ]:
pred = model2.predict(X_test_seq) 
pred = np.argmax(pred, axis = 1)[:5] 
label = np.argmax(y_test_seq,axis = 1)[:5] 

print(pred) 
print(label)

In [ ]:
sentiment = ['Negative','Neutral','Positive']
sequence = tokenizer.texts_to_sequences(['mal curso'])
print(sequence)
#test = sequence.pad_sequences(sequence, maxlen=100)
sentiment[np.around(model2.predict(sequence), decimals=0).argmax(axis=1)[0]]
print(np.around(model2.predict(sequence), decimals=0).argmax(axis=1)[0])
print(sentiment[np.around(model2.predict(sequence), decimals=0).argmax(axis=1)[0]])

[[472, 30]]
2
Positive
